
# Creating entangled states with optimized circuits for particular backend


## Introduction

Our goal is to entangle all the qubits available in a given backend. Entanglement in one of the most important properties of quantum systems in quite many quantum algorithms and protocols and, therefore, we find it a suited choise for our task.
It also motivates us to learn more about the quantum hardware on which the actual state is created in order to optimize the circuit correlated with the physical architecture and to mitigate the noise contribution.

## Idea

The first approach it was to create a GHZ state applying a hadamard gate on the first qubit and entangling afterwise the others one by one, resulting in a O(N) evolution regarding the number of qubits. Our solution is based on paralelization of the entanglement process taking into consideration the architecture of the given quantum chip and the quality of the interqubit connections. We defined the quantum circuit as a graph and optimized it to minimize the circuit depth.

## Solution

In [6]:
## Importing the necesary libraries
from qiskit import QuantumCircuit
from qiskit.providers.fake_provider import FakePrague, FakeSherbrooke, FakeAuckland , FakeQuitoV2
from qiskit import transpile
from qiskit.tools.visualization import plot_histogram
from qiskit.quantum_info import entropy, Statevector
from rustworkx.visualization import graphviz_draw
import rustworkx as rw
from rustworkx import minimum_spanning_edges, PyGraph, dijkstra_shortest_path_lengths
import matplotlib.pyplot as plt
import math
from random import randrange

## Naive Approach

We will take the backend with a semnificative number of qubits, in order to have a statistically significant result.

In [7]:
my_variables = set(dir())


# Get fake backends from the fake providers
prague = FakePrague()
sherbrooke = FakeSherbrooke()
auckland = FakeAuckland()
quitov2 = FakeQuitoV2()


# Create a simple GHZ circuit to entangle all the qubits in the system
num_qubits_sherbrooke = sherbrooke.num_qubits
circuit = QuantumCircuit(num_qubits_sherbrooke)
circuit.h(0)
for i in range(num_qubits_sherbrooke - 1):
    circuit.cx(i, i+1)

# this circuit is way too large to be shown in a figure

naive_depth_sherbrooke = circuit.depth()
print(naive_depth_sherbrooke)

127


# Solution

In [8]:
# Define the optimizing algorithm 
def get_optimized_state_random(backend):
    # For V1 backends you should consult the documentation for the right attribute
    num_qubits = backend.num_qubits
    connections = backend.coupling_map
    
    # Transpile the ideal circuit to a circuit that can be directly executed by the backend
    qc = QuantumCircuit(num_qubits)
    transpiled_circuit = transpile(qc, backend)
    transpiled_circuit.draw('mpl')

    # We create a dictionary for every qubit in order to label them as targeted and controls
    # In every round it is applied a controlled not gate from the control qubits to the targets and then 
    # recursevly every target node becomes a control node untill there are no more unentangled qubits
    def non_entangled_exist(target_dict):
        for i in range(len(target_dict)):
            if target_dict[i] == 'n':
                return True
        return False

    targeted = {}
    for i in range(num_qubits):
        targeted.update({i: 'n'})
    start = randrange(num_qubits)
    print("Start qubit: ", start)

    qc.h(start)
    # mark this qubit as 'targeted'
    targeted[start] = 't'

    list_ctrl = []
    list_ctrl.append(start)

    iteration = 0
    while non_entangled_exist(targeted):
        iteration += 1
        # helper list_ctrl to be used for swap at the end of the loop
        swp_list_ctrl = []
        # for each node in list_ctrl find connected nodes which haven't been entangled (targeted)
        for i in list_ctrl:
            # find connected nodes in connections not targeted
            for j in connections:
                # apply cx on it
                # mark it as targeted
                # add it to swp_list_ctrl
                # (configuration map contains redundant connections both ways between nodes, i.e. [i,j] == [j,i])
                if j[0] == i:
                    if targeted[j[1]] == 'n':
                        qc.cx(i, j[1])
                        targeted[j[1]] = 't'
                        swp_list_ctrl.append(j[1])
                if j[1] == i:
                    if targeted[j[0]] == 'n':
                        qc.cx(i, j[0])
                        targeted[j[0]] = 't'
                        swp_list_ctrl.append(j[0])
        # clear list_ctrl
        list_ctrl = []
        # replace list_ctrl with swp_list_ctrl
        for i in swp_list_ctrl:
            list_ctrl.append(i)
        qc.barrier()


    return [qc.depth(), num_qubits]

# Testing the solution on the Sherbrooke backend with a random initial state

print("The circuit depth: ", get_optimized_state_random(sherbrooke)[0])

Start qubit:  117
The circuit depth:  37


## Selecting a the qubit corresponding to the root node in the spanning tree

In [9]:
def get_optimized_state_root(backend):
    # For V1 backends you should consult the documentation for the right attribute
    num_qubits = backend.num_qubits
    connections = backend.coupling_map
    
    # Transpile the ideal circuit to a circuit that can be directly executed by the backend
    qc = QuantumCircuit(num_qubits)
    transpiled_circuit = transpile(qc, backend)
    transpiled_circuit.draw('mpl')

    # Defining the metrics for the connections between the qubits
    def edge_cost_fn(weight):
        return math.exp(weight)
        # return weight
    
    # We create a graph to map the qubits physical architecture mimicing the calibration with the weights of the edges
    graph = rw.PyGraph()
    edges = connections.get_edges()
    weighted_edges = [(source, target, randrange(1,100)/1000) for source, target in edges]
    graph.extend_from_weighted_edge_list(weighted_edges)
    
    # We find equivalent tree reprezentation and choose the root node in such a way that it has the minimum distance to
    # the furthest leave 
    min_max_length = float('inf')
    centroid = None
    for node in graph.node_indices():
        lengths = dijkstra_shortest_path_lengths(graph, node, edge_cost_fn)
        max_length = max(lengths.values())
        if max_length < min_max_length:
            min_max_length = max_length
            centroid = node

    # We create a dictionary for every qubit in order to label them as targeted and controls
    # In every round it is applied a controlled not gate from the control qubits to the targets and then 
    # recursevly every target node becomes a control node untill there are no more unentangled qubits
    def non_entangled_exist(target_dict):
        for i in range(len(target_dict)):
            if target_dict[i] == 'n':
                return True
        return False
   
    targeted = {}
    for i in range(num_qubits):
        targeted.update({i: 'n'})
    start = centroid
    print("Start qubit: ", start)

    qc.h(start)
    # print(qc.draw())
    # mark this qubit as 'targeted'
    targeted[start] = 't'

    list_ctrl = []
    list_ctrl.append(start)

    iteration = 0
    while non_entangled_exist(targeted):
        iteration += 1
        # helper list_ctrl to be used for swap at the end of the loop
        swp_list_ctrl = []
        # for each node in list_ctrl find connected nodes which haven't been entangled (targeted)
        for i in list_ctrl:
            # find connected nodes in connections not targeted
            for j in connections:
                # apply cx on it
                # mark it as targeted
                # add it to swp_list_ctrl
                if j[0] == i:
                    if targeted[j[1]] == 'n':
                        qc.cx(i, j[1])
                        targeted[j[1]] = 't'
                        swp_list_ctrl.append(j[1])
                if j[1] == i:
                    if targeted[j[0]] == 'n':
                        qc.cx(i, j[0])
                        targeted[j[0]] = 't'
                        swp_list_ctrl.append(j[0])
        # clear list_ctrl
        list_ctrl = []
        # replace list_ctrl with swp_list_ctrl
        for i in swp_list_ctrl:
            list_ctrl.append(i)
        qc.barrier()  

    return [qc.depth(), num_qubits]

# Testing the solution on the Sherbrooke fake backend with the starting qubit as the root of the spanning tree
print("The circuit depth: ", get_optimized_state_root(sherbrooke)[0])

Start qubit:  63
The circuit depth:  22


# Results

In [16]:
# Comparing the results on different face backend
backends = [ prague, quitov2, auckland, sherbrooke ]
bks = [ 'prague', 'quitov2', 'auckland', 'sherbrooke' ]
for backend, bk in zip(backends, bks):
    result = get_optimized_state_root(backend)
    percentage = (1-result[0]/result[1])
    print(f'Depth reduction (percents): {bk} --> {percentage:.3}')


Start qubit:  13
Depth reduction (percents): prague --> 0.606
Start qubit:  1
Depth reduction (percents): quitov2 --> 0.0
Start qubit:  13
Depth reduction (percents): auckland --> 0.593
Start qubit:  64
Depth reduction (percents): sherbrooke --> 0.819
